# Convert proposed road speed rasters to access surfaces

This notebook merges each of the new speed rasters to the "main" friction surface for the analysis extent area, generating a new friction surface specific to that road. We use that friction surface to generate accessibility surfaces specific to that road

In [2]:
import os, sys
from datetime import date

import numpy as np
import re

import rasterio
from rasterio import features, transform
from rasterio.merge import merge as merge
from rasterio.transform import Affine
from rasterio.io import MemoryFile

import rio_cogeo
from rio_cogeo.cogeo import cog_translate

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import shape, box, MultiPoint, Point, Polygon

import skimage.graph as graph
sys.path.append('../../src/')
import GOSTnets.GOSTNets as gn
import GOSTNets_Raster.src.GOSTNets_Raster.market_access as ma
from gostrocks.src.GOSTRocks.misc import tPrint

## Setup

Dates

In [3]:
today = date.today().strftime("%d%m%y")

In [4]:
data_date = '211020'

Directories

In [5]:
geo_dir = r'P:\PAK\GEO'
data_dir = r'../../data'

rd_dir = r'roads'
dest_dir = r'destinations'
fric_dir = r'friction'
acc_dir = r'access'

Projections

In [6]:
# change this to whatever the desired output projection is
dest_crs = 'EPSG:32642'

Desired resolution of rasters

In [7]:
res = '31m'

Load in KP as clipping object

In [8]:
aoi = gpd.read_file(os.path.join(geo_dir,'Boundaries/OCHA/pak_admbnda_adm1_ocha_pco_gaul_20181218.shp'))
aoi = aoi[aoi['ADM1_EN'] == 'Khyber Pakhtunkhwa']
aoi = aoi.to_crs(dest_crs)

# Buffer the polygon by 20km so we take in nearby markets and roads that may be used
aoi.geometry = aoi.buffer(20000)

Destination files

In [9]:
# Use destinations prepared previously. This assumes they are in WGS84 and need to be reprojected to the project's metric CRS

dest_fils = {
    re.findall(r'KP_(.*?).gpkg',fil)[0]: gpd.clip(gpd.read_file(os.path.join(data_dir,dest_dir,fil))\
                                                  .set_crs(4326).to_crs(dest_crs),aoi) \
    for fil in os.listdir(os.path.join(data_dir,dest_dir)) if fil.endswith(".gpkg")
}

In [10]:
len(dest_fils)

24

Slight adjustment to GOST's TT code: export rasters as Float32 to reduce file sizes

In [11]:
# adjust GOST's code to always export float32 rasters, as a space saving measure

def calculate_travel_time_slim(inH, mcp, destinations, out_raster = ''):
    ''' Calculate travel time raster
    
    INPUTS
        inH [rasterio object] - template raster used to identify locations of destinations
        mcp [skimage.graph.MCP_Geometric] - input graph
        destinations [geopandas df] - destinations for nearest calculations
        
    LINKS
        https://scikit-image.org/docs/0.7.0/api/skimage.graph.mcp.html#skimage.graph.mcp.MCP.find_costs
    '''
    # create skimage graph, force costs to float32 to save on space
    cities = ma.get_mcp_dests(inH, destinations)
    costs, traceback = mcp.find_costs(cities)
    costs = costs.astype(np.float32)
    if not out_raster == '':
        meta = inH.meta.copy()
        meta.update(dtype=costs.dtype)
        with rasterio.open(out_raster, 'w', **meta) as out:
            out.write_band(1, costs)
            
    return((costs, traceback))

## Generate friction surfaces

Create lists of variable names to loop over

In [12]:
speed_cols = ['upgrade_dry_speed', 'upgrade_msn_speed', 'upgrade_winter_speed']

In [13]:
seasons = ['dry','msn','winter']

**Create a list of friction tif file paths** for roads that have not yet been computed into all-of-KP friction surfaces

In [ ]:
# friction surfaces
fric_tifs = [os.path.join(data_dir,fric_dir,f'proposed_roads//{res}//raw',file) \
            for file \
            in os.listdir(os.path.join(data_dir,fric_dir,f'proposed_roads//{res}//raw',)) \
            if file.endswith(".tif")]


If doing this in multiple batches, use the below code. This is useful if you don't recieve all your roads files at once or find errors in some that need to be subsequently corrected.

In [ ]:
# # create a list of already completed friction surfaces to exclude, so we don't waste time recreating data

# exclusion_lst = sorted(list(set([re.search('^([0-9]+)',file)[0] \
#             for file \
#             in os.listdir(os.path.join(data_dir,fric_dir,f'proposed_roads//{res}//allKP')) \
#             if file.endswith(".tif")]))) # change to whatever criteria you need

# # filter down friction tifs using the exclusion list

# fric_tifs = [file for file in fric_tifs \
#               if re.search('^[0-9]+',os.path.basename(file))[0] not in exclusion_lst]

In [ ]:
fric_tifs[::10]

#### Merge files, convert to friction surfaces if necessary, and export

In [ ]:
# create export dir if missing
if os.path.exists(os.path.join(data_dir,fric_dir,f'proposed_roads//{res}//allKP')) == False:
    os.mkdir(os.path.join(data_dir,fric_dir,f'proposed_roads//{res}//allKP'))
else:
    None

In [ ]:
for seas in seasons:
    
    # name variables
    base_fric_rast_pth = f'KP_friction_{seas}_{data_date}_{res}_masked_final.tif'
    
    # load in base raster as DatasetReader
    base_fric_rast = rasterio.open(os.path.join(data_dir,fric_dir,f'{res}//current_w_proposed',base_fric_rast_pth),'r')
    base_profile = base_fric_rast.profile
    base_profile.update({'dtype':'float32'})
    
    # if merging to a friction surface

    for idx, tif in enumerate(fric_tifs):
    
        if seas in tif:
            
            # create variable for naming output file
            feat_name = os.path.splitext(os.path.basename(fric_tifs[idx]))[0]

            # countdown
            tPrint(f'{seas}, {idx + 1} of {len(fric_tifs)}, {feat_name}')

            # merge two raster datasets, always taking the lowest value (friction cost)
            merge_array, out_transform = merge([base_fric_rast,rasterio.open(tif,'r')],method='min')
            merge_array = merge_array.astype(np.float32)

            # export result
            out_name = f'{feat_name}_{res}.tif'.replace(' ','')

            with rasterio.open(
                os.path.join(data_dir,fric_dir,f'proposed_roads//{res}//allKP',out_name), 'w',**base_profile) as dst:
                dst.write(merge_array)
                
        else:
            None

## Generate access surfaces to all destinations, for all roads, for every season

Roads

In [14]:
master_prop_rds = gpd.read_file(os.path.join(data_dir,rd_dir,f'Proposed_final//Proposed_roads_processed_{data_date}.gpkg'),driver="GPKG")

Populate a list of friction surface paths for roads that have not yet been computed into access surfaces

In [21]:
# friction surfaces
complete_fric_tifs = sorted([os.path.join(data_dir,fric_dir,f'proposed_roads//{res}//allKP',file) \
            for file \
            in os.listdir(os.path.join(data_dir,fric_dir,f'proposed_roads//{res}//allKP')) \
            if file.endswith(".tif")])


If doing this in multiple batches, use the below code.

In [ ]:
# # create a list of already completed friction surfaces to exclude, so we don't waste time recreating data
# exclusion_lst = sorted(list(set([re.search('^([0-9]+)',file)[0] \
#             for file \
#             in os.listdir(os.path.join(data_dir,acc_dir,f'upgrade//{res}')) \
#             if 'dry' in file]))) # currently set up for walking rasters, change to whatever you need
#             if file.endswith(".tif")]))) # currently set up for walking rasters, change to whatever you need

# # filter down friction tifs using the exclusion list
# complete_fric_tifs = [file for file in complete_fric_tifs \
#               if re.search('^[0-9]+',os.path.basename(file))[0] not in exclusion_lst]

In [24]:
complete_fric_tifs[::10]

['../../data\\friction\\proposed_roads//31m//allKP\\10_LowerChitral_upgrade_dry_friction_31m.tif',
 '../../data\\friction\\proposed_roads//31m//allKP\\20_Hangu_upgrade_dry_friction_31m.tif',
 '../../data\\friction\\proposed_roads//31m//allKP\\35_Karak_upgrade_dry_friction_31m.tif',
 '../../data\\friction\\proposed_roads//31m//allKP\\53_Laki_upgrade_dry_friction_31m.tif',
 '../../data\\friction\\proposed_roads//31m//allKP\\70_Bajaur_upgrade_dry_friction_31m.tif',
 '../../data\\friction\\proposed_roads//31m//allKP\\81_Khuber_upgrade_dry_friction_31m.tif']

Generate access surfaces in a loop</br>**Warning**: This step is slow at large sizes! Run overnight or over a weekend even

In [ ]:
# create export dir if missing
if os.path.exists(os.path.join(data_dir,acc_dir,f'upgrade//{res}')) == False:
    os.mkdir(os.path.join(data_dir,acc_dir,f'upgrade//{res}'))
else:
    None

In [ ]:
# Loop to generate clipped access surfaces for all roads, in all seasons, to all destinations

for dest_idx, (dest_name, dest_gdf) in enumerate(dest_fils.items()):
    
    # countdown
    tPrint(f'{dest_idx + 1} of {len(dest_fils)} destinations: {dest_name}')
    
    for fric_idx, fric_tif_fil in enumerate(complete_fric_tifs):
        
        # important variables for filtering and naming
        
        season = re.findall(r'upgrade_(.*?)_friction',fric_tif_fil)[0]
        feat_name = os.path.splitext(os.path.basename(complete_fric_tifs[fric_idx]))[0].split('_friction')[0]
        sn = int(re.split(r'([0-9]+)',os.path.basename(complete_fric_tifs[fric_idx]))[1])
        
        # countdown
        tPrint(f'{fric_idx + 1} of {len(complete_fric_tifs)}, {dest_name}, {feat_name}')
        
        # create a bounding box for clipping all our inputs to the focus area for each road
        # the bbox extent = the distance to the Nth nearest destination + the length of the road + half a kilometer, as previously prepared

        buf_dist = master_prop_rds[master_prop_rds['SN'] == sn][f'{dest_name}_nth_nearest_distance'].values[0]
        bds = master_prop_rds[master_prop_rds['SN'] == sn].geometry.total_bounds
        bbox = box(bds[0],bds[1],bds[2],bds[3]).buffer(buf_dist)

        # clip friction surface
    
        with rasterio.open(fric_tif_fil) as fric_raw:
            fric_clip, fric_clip_tform = rasterio.mask.mask(fric_raw,[bbox],crop=True)
            fric_clip = fric_clip[0,:,:]
            fric_clip_profile = fric_raw.profile
            
            # create mcp object from clipped friction surface

            inD = np.nan_to_num(fric_clip,nan=10,posinf=10)
            mcp = graph.MCP_Geometric(inD)

        # update profile with new shape and tform

        fric_clip_profile.update({"height":fric_clip.shape[0],
                             "width":fric_clip.shape[1],
                             "transform" : fric_clip_tform})

        # Create a clipped raster in memory
        # This routine is a bit different from how this is done (via the clip_in_memory custom function) elsewhere in these notebooks as here we only seek a DatasetReader object, meaning the raster is not yet converted to a numpy array
        with MemoryFile() as memfile:
            with memfile.open(**fric_clip_profile) as dataset:
                dataset.write(fric_clip,indexes=1)

            # now re-open the memfile to create a DatasetReader object for the calculate_travel_times routine
            memfile.seek(0)
            fric_clip_in_mem = memfile.open()

        # clip input gdf
        
        clipped_gdf = gpd.clip(dest_gdf,bbox).set_crs(dest_crs)
            
        # compute costs and export
        # Note that costs around the margins of the clipped surface will be HIGHER because low-cost roads beyond the edges are missing. 
        # This is a byproduct of clipping (needed for space + time) that we'll sort in the next notebook
        
        costs, traceback = calculate_travel_time_slim(fric_clip_in_mem, mcp, clipped_gdf, \
                                                    os.path.join(data_dir,acc_dir,f'upgrade//{res}//{feat_name}_{dest_name}_access.tif'))

07:32:47	1 of 24 destinations: District_HQs
07:32:47	1 of 59, District_HQs, 10_LowerChitral_upgrade_dry
07:34:17	2 of 59, District_HQs, 11_LowerChitral_upgrade_dry
07:36:52	3 of 59, District_HQs, 12_Kohistan_upgrade_dry
07:38:54	4 of 59, District_HQs, 13_Kohistan_upgrade_dry
07:39:44	5 of 59, District_HQs, 14_Kohistan_upgrade_dry
07:40:29	6 of 59, District_HQs, 15_Kohistan_upgrade_dry
07:41:10	7 of 59, District_HQs, 17_Torghar_upgrade_dry
07:41:40	8 of 59, District_HQs, 18_Torghar_upgrade_dry
07:42:39	9 of 59, District_HQs, 19_Hangu_upgrade_dry
07:43:03	10 of 59, District_HQs, 1_UpperChitral_upgrade_dry
07:46:40	11 of 59, District_HQs, 20_Hangu_upgrade_dry
07:47:51	12 of 59, District_HQs, 21_Hangu_upgrade_dry
07:48:29	13 of 59, District_HQs, 22_Hangu_upgrade_dry
07:49:02	14 of 59, District_HQs, 23_Hangu_upgrade_dry
07:49:23	15 of 59, District_HQs, 24_Karak_upgrade_dry
07:50:22	16 of 59, District_HQs, 25_Karak_upgrade_dry
07:51:43	17 of 59, District_HQs, 26_Karak_upgrade_dry
07:52:27	18

All done. Move on to the next step.